# Lab 5: Merging & Relational Databases

<a href="https://colab.research.google.com/github/gaulinmp/AccountingDataAnalytics/blob/main/labs_hw/week5_RDB/Lab%205%20-%20Merging%20%26%20Relational%20Databases.ipynb" target="_parent">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

1. Connect to the PostgreSQL database
2. Explore the CRSP data structure, identifying what stock related variables are available
3. Download daily returns data for the tickers `AAPL`, `MSFT`, and `GOOGL`
4. Aggregate daily returns to monthly frequency
5. Create visualizations of daily returns, monthly returns, and trading volume
   1. Line chart of daily returns for the 3 tickers over at least 5 years (starting January 2020 or earlier)
   2. Line chart of daily trading volume for the same tickers
   3. Line chart of monthly returns for the same 3 tickers over the same time period
   4. Line chart of the monthly trading volume for the same tickers

## Step 1: Import Libraries and Connect to Database

The code below imports the required libraries and sets up some code that will connect to the PostgreSQL database. 
Make sure to replace the placeholder values with the actual connection details provided in Canvas.

The function below, `read_sql`, will wrap all the code needed to connect to the database, run a SQL query, and return the results as a pandas DataFrame.
So all you have to do is `df = read_sql("SELECT * FROM crsp_daily WHERE ticker IN ('AAPL', 'MSFT', 'GOOGL')")` to get the data you need.

In [ ]:
!pip install psycopg

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from contextlib import contextmanager
from sqlalchemy import create_engine, inspect, text
from sqlalchemy.orm import sessionmaker
import urllib

db_name = 'ADA_SQL'
db_host = "COPY FROM CANVAS"
db_user = 'student'
db_password = "COPY FROM CANVAS"
db_port = '5432'

db_url = f"postgresql+psycopg://{db_user}:{urllib.parse.quote(db_password)}@{db_host}:{db_port}/{db_name}"

@contextmanager
def session_scope(debug=False):
    global db_engine, db_url
    db_engine = create_engine(db_url, pool_pre_ping=True, echo=debug)
    Session = sessionmaker(bind=db_engine)
    session = Session()
    try:
        # this is where the "work" happens!
        yield session
        # always commit changes!
        session.commit()
    except:
        # if any kind of exception occurs, rollback transaction
        session.rollback()
        raise
    finally:
        session.close()
        db_engine.dispose()

def read_sql(query):
    with session_scope() as session:
        return pd.read_sql(query, session.bind)

## Step 2: Extract the data from the database

I've wrapped the annoying part of connecting to the database and running a SQL query into the `read_sql` function above.
So you can just play around with SQL queries and call `read_sql` to get the data you need.

In [ ]:
try:
    # Check data coverage and structure
    query = """
    SELECT DISTINCT
        MIN(date) as min_date,
        MAX(date) as max_date,
        COUNT(DISTINCT ticker) as num_companies,
        COUNT(*) as total_records
    FROM crsp_daily;
    """

    display(read_sql(query))
except Exception as e:
    print(f"💀 Cannot explore data - no database connection. Error: {e}")

In [ ]:
try:
    sql_query = """
    SELECT *
    FROM crsp_daily
    WHERE ticker IN ('AAPL', 'MSFT', 'GOOGL')
    AND date >= '2020-01-01'
    ORDER BY ticker, date;
    """

    df_raw = read_sql(sql_query)
    print(f"🥳 Successfully read {len(df_raw):,} rows from the database")
except Exception as e:
    print(f"💀 Cannot explore data - no database connection. Error: {e}")

## Step 3: Clean and Describe the data

In [ ]:
# It's good to keep a raw version of the data, so if you mess something up, you can always go back to the original
df = df_raw.copy()
# Look at the first few rows
df.head()

In [ ]:
# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# Look at summary statistics
df.describe(include='all')

In [ ]:
# Add monthly returns
df_monthly = (
    df
    # groupby can take column names, but can also just take columns themselves
    .groupby([df.ticker, df.date.dt.to_period("M")])
    # Agg calculates the following columns, where it's :
    # new column name=(column to calculate, calculation function)
    .agg(
        cumulative_return=('ret', lambda x: (1 + x).prod() - 1),  # Compound returns
        ave_ret=('ret', 'mean'),  # Average returns
        vol=('vol', 'sum'),  # Total monthly volume
        date_last_of_month=('date', 'last'),  # End of month date
        shrout=('shrout', 'last'),  # End of month shares outstanding
        prc=('prc', 'last')   # End of month price
    )
    .reset_index()
)

df_monthly.head()

In [ ]:
df_monthly.describe(include='all').T

## Step 4: Create Charts

In [ ]:
# Chart 1. Line chart of daily returns for the 3 tickers over at least 5 years (starting January 2020 or earlier)
sns.lineplot(data=df, x='CHOOSE X VARIABLE', y='CHOOSE Y VARIABLE', hue='HUE WILL MAKE SEPARATE LINES')
plt.savefig('chart1_daily_returns.png', dpi=300)

In [ ]:
# Chart 2. Line chart of daily trading volume for the same tickers
sns.lineplot(data=df, x='CHOOSE X VARIABLE', y='CHOOSE Y VARIABLE', hue='HUE WILL MAKE SEPARATE LINES')
plt.savefig('chart2_daily_volume.png', dpi=300)

In [ ]:
# Chart 3. Line chart of monthly returns for the same 3 tickers over the same time period
sns.lineplot(data=df_monthly, x='CHOOSE X VARIABLE', y='CHOOSE Y VARIABLE', hue='HUE WILL MAKE SEPARATE LINES')
plt.savefig('chart3_monthly_returns.png', dpi=300)

In [ ]:
# Chart 4. Line chart of the monthly trading volume for the same tickers
sns.lineplot(data=df_monthly, x='CHOOSE X VARIABLE', y='CHOOSE Y VARIABLE', hue='HUE WILL MAKE SEPARATE LINES')
plt.savefig('chart4_monthly_volume.png', dpi=300)